In [72]:
import numpy as np
from Network import Network # neural network class, defined in separate file
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.linear_model import MultiTaskLassoCV, RidgeCV
%matplotlib inline

# Exercise 1: Logistic Regression and perceptron

In [2]:
wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None).values
X = wine[:, 1:]
y = OneHotEncoder(categories="auto").fit_transform(wine[:, 0].reshape(-1, 1)).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### SGD vs GD for multiclass logistic regression

In [60]:
learning_rate = 1
epochs = 250

df_accuracy = pd.DataFrame(columns=["GD train accuracy", "GD test accuracy", "SGD train accuracy", "SGD test accuracy"])
df_auc = pd.DataFrame(columns=["GD train auc", "GD test auc", "SGD train auc", "SGD test auc"])

for i in range(5):
    network_gd = Network([X.shape[1], y.shape[1]], loss="mse", learning_rate=learning_rate)
    network_sgd = Network([X.shape[1], y.shape[1]], loss="mse", learning_rate=learning_rate)
    
    network_gd.train(X_train, y_train, X_test, y_test, epochs=epochs, sgd=False)
    network_sgd.train(X_train, y_train, X_test, y_test, epochs=epochs, sgd=True)
    
    df_accuracy.loc[len(df_accuracy)] = [
        network_gd.accuracy(X_train, y_train),
        network_gd.accuracy(X_test, y_test),
        network_sgd.accuracy(X_train, y_train),
        network_sgd.accuracy(X_test, y_test)]
    
    df_auc.loc[len(df_accuracy)] = [
        roc_auc_score(y_train, network_gd.predict_proba(X_train)),
        roc_auc_score(y_test, network_gd.predict_proba(X_test)),
        roc_auc_score(y_train, network_sgd.predict_proba(X_train)),
        roc_auc_score(y_test, network_sgd.predict_proba(X_test))]
         

In [66]:
print(np.mean(df_accuracy))

GD train accuracy    0.9698
GD test accuracy     0.9278
SGD train accuracy   0.9736
SGD test accuracy    0.9278
dtype: float64


In [67]:
df_auc

,GD train auc,GD test auc,SGD train auc,SGD test auc
1,0.9974,0.9923,0.9953,0.9903
2,0.9995,0.9947,0.9965,0.9916
3,0.9965,0.9960,0.9992,0.9905
4,0.9979,0.9915,0.9992,0.9971
5,0.9978,0.9964,0.9952,0.9884


### Ridge and Lasso

In [97]:
df_acc2 = pd.DataFrame(columns=["Lasso train accuracy", "Lasso test accuracy", 
                                "Ridge train accuracy", "Ridge test accuracy"])

for i in range(5):
    lasso = MultiTaskLassoCV(verbose=0)
    lasso.fit(X_train, y_train)

    ridge = RidgeCV()
    ridge.fit(X_train, y_train)
    
    df_acc2.loc[len(df_acc2)] = [
        accuracy_score(np.argmax(lasso.predict(X_train),axis=1), np.argmax(y_train, axis=1)),
        accuracy_score(np.argmax(lasso.predict(X_test),axis=1), np.argmax(y_test, axis=1)),
        accuracy_score(np.argmax(ridge.predict(X_train),axis=1), np.argmax(y_train, axis=1)),
        accuracy_score(np.argmax(ridge.predict(X_test),axis=1), np.argmax(y_test, axis=1))]

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning:

The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning:

The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning:

The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning:

The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning:

The default value of cv will change from 3 to 5 in ve

In [98]:
print(np.mean(df_acc2))

Lasso train accuracy   1.0000
Lasso test accuracy    0.9306
Ridge train accuracy   1.0000
Ridge test accuracy    0.9306
dtype: float64


### Step vs logistic

In [99]:
learning_rate = 1
epochs = 250

df_acc3 = pd.DataFrame(columns=["Sigmoid train accuracy", "Sigmoid test accuracy", 
                                "Step train accuracy", "Step test accuracy"])

for i in range(5):
    network_sigmoid = Network([X.shape[1], y.shape[1]], loss="mse", learning_rate=learning_rate)
    network_sigmoid.train(X_train, y_train, X_test, y_test, epochs=epochs, sgd=True)
    
    network_step = Network([X.shape[1], y.shape[1]], loss="mse", learning_rate=learning_rate, 
                           output_activation="step")
    network_step.train(X_train, y_train, X_test, y_test, epochs=epochs, sgd=True)
    
    df_acc3.loc[len(df_acc3)] = [
        network_sigmoid.accuracy(X_train, y_train),
        network_sigmoid.accuracy(X_test, y_test),
        network_step.accuracy(X_train, y_train),
        network_step.accuracy(X_test, y_test)]

In [101]:
print(np.mean(df_acc3))

Sigmoid train accuracy   0.9717
Sigmoid test accuracy    0.8889
Step train accuracy      0.9962
Step test accuracy       0.8611
dtype: float64


# Exercise 2: Feed Forward Neural Network

### Performance on iris dataset

In [ ]:
iris = datasets.load_iris()
X = iris.data
X = StandardScaler().fit_transform(X)
y_raw = iris.target
y = OneHotEncoder(categories="auto").fit_transform(y_raw.reshape(-1, 1)).toarray()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [ ]:
network = Network([4, 5, 5, 3], 
                  dense_activation="tanh", output_activation="sigmoid", 
                  loss="log_loss", learning_rate=0.5)

In [ ]:
train_loss, train_accuracy, val_loss, val_accuracy = network.train(X_train, y_train, 
                                                                   X_test, y_test, epochs=400)

In [ ]:
plt.figsize=(12, 8)
plt.title("Loss")
plt.plot(train_loss, label="train loss")
plt.plot(val_loss, label="val loss")
plt.legend()

In [ ]:
plt.figsize=(12, 8)
plt.title("Accuracy")
plt.plot(train_accuracy, label="train accuracy")
plt.plot(val_accuracy, label="val accuracy")
plt.legend()

### Performance on wine dataset

In [ ]:
wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None).values
X = wine[:, 1:]
y = OneHotEncoder(categories="auto").fit_transform(wine[:, 0].reshape(-1, 1)).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
network = Network([13, 8, 8, 3], 
                  dense_activation="tanh", output_activation="sigmoid", 
                  loss="log_loss", learning_rate=0.5)

In [ ]:
train_loss, train_accuracy, val_loss, val_accuracy = network.train(X_train, y_train, 
                                                                   X_test, y_test, epochs=400)

In [ ]:
plt.figsize=(12, 8)
plt.title("Loss")
plt.plot(train_loss, label="train loss")
plt.plot(val_loss, label="val loss")
plt.legend()

In [ ]:
plt.figsize=(12, 8)
plt.title("Accuracy")
plt.plot(train_accuracy, label="train accuracy")
plt.plot(val_accuracy, label="val accuracy")
plt.legend()